# Hyperparameter Tuning Example

In this example, we'll use sklearn's `GridSearchCV` class to find
optimal hyperparameters of the `RandomForestClassifier` estimator.

In [1]:
# Load the library with the iris dataset
from sklearn.datasets import load_iris

# Load scikit's random forest classifier library
from sklearn.ensemble import RandomForestClassifier

# Load pandas
import pandas as pd

# Load numpy
import numpy as np

# Set random seed
np.random.seed(0)

# train_test_split: for splitting data into training and test datasets
# GridSearchCV: for hyperparameter tuning
from sklearn.model_selection import train_test_split, GridSearchCV

# Evaluate our results
from sklearn.metrics import classification_report, accuracy_score

In [2]:
import warnings
warnings.filterwarnings('ignore')

### Prepare the data

Since the `load_iris()` function returns the data as numpy matrices already, we
won't be dealing with any Pandas data frames just yet. If we were starting with
Pandas, we'd need to convert to numpy matrices first.

In [3]:
# Create an object called iris with the iris data
iris = load_iris()

In [4]:
# Extract the predictor attributes
X = iris.data

# Extract the target attribute
y = iris.target.reshape(-1,1)

In [5]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.333)

In [6]:
# Display shape of the new datasets
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

X_train: (100, 4)
y_train: (100, 1)
X_test: (50, 4)
y_test: (50, 1)


### Conduct hyperparameter tuning

We must be careful NOT TO USE the test dataset during this process!

In [7]:
# Configure the parameters to be tuned
tuned_parameters = {
    'n_estimators': range(20,101,5),
    'max_depth': range(20,61,5),
    'n_estimators': range(10,41,5),
    'max_features': ['sqrt', 'log2', None]
}

In [8]:
# Initialize a GridSearchCV object
clf = GridSearchCV(
    RandomForestClassifier(),
    tuned_parameters,
    cv=5, # 5-fold cross-validation
    scoring='precision_weighted')

In [9]:
%%time
# Conduct the tuning. This may take a little while.
clf.fit(X_train, y_train.ravel())

CPU times: user 19.5 s, sys: 103 ms, total: 19.6 s
Wall time: 19.8 s


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': range(10, 41, 5), 'max_depth': range(20, 61, 5), 'max_features': ['sqrt', 'log2', None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='precision_weighted', verbose=0)

In [10]:
cv_results = pd.DataFrame(clf.cv_results_)
print("We trained {} models.".format(len(cv_results)))

We trained 189 models.


In [11]:
print("The best model gave score of {:.3f}".format(clf.best_score_))
print("Parameters:", clf.best_params_)

The best model gave score of 0.967
Parameters: {'max_depth': 20, 'max_features': 'log2', 'n_estimators': 20}


In [12]:
print("Here are the details of that job:")
print(cv_results.loc[clf.best_index_])

Here are the details of that job:
mean_fit_time                                                 0.0134144
std_fit_time                                                 0.00180222
mean_score_time                                              0.00189981
std_score_time                                              0.000187811
param_max_depth                                                      20
param_max_features                                                 log2
param_n_estimators                                                   20
params                {'max_depth': 20, 'max_features': 'log2', 'n_e...
split0_test_score                                              0.925926
split1_test_score                                                     1
split2_test_score                                                     1
split3_test_score                                               0.95625
split4_test_score                                              0.953947
mean_test_score               

### Estimate how well this model will generalize to unseen data
We'll do this by making predictions on the test set.

In [13]:
# Let's now estimate generalization error by
# using our optimal model to make predictions on the test set.
y_pred = clf.predict(X_test)

In [14]:
# Classification report (rows are the target classes 0, 1, 2)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       0.95      0.95      0.95        19
           2       0.93      0.93      0.93        15

   micro avg       0.96      0.96      0.96        50
   macro avg       0.96      0.96      0.96        50
weighted avg       0.96      0.96      0.96        50



In [15]:
# How well will this model generalize to unseen data (in terms of overall accuracy)?
print('{:.3f}'.format(accuracy_score(y_test, y_pred)))

0.960
